<center><img src="https://storage.googleapis.com/arize-assets/arize-logo-white.jpg" width="200"/></center>

# Supply Chain Optimization in the Manufacturing Industry
Walk through how to use Arize for a supply chain optimization application using an example dataset.

## 📨 Upload Data to Arize
Upload example data to Arize, this example uses the [Python Pandas method](https://docs.arize.com/arize/sending-data-methods/log-directly-via-sdk-api).

In [ ]:
# Install and import dependencies

!pip install -q arize
from arize.pandas.logger import Client, Schema
from arize.utils.types import ModelTypes, Environments, Metrics

import pandas as pd
import numpy as np

### 🌐 Upload Data to Arize: Download Data
Here are sample parquet files that represent the <strong>training</strong> and <strong>production</strong> data of a model designed to forecast product demand to aid supply chain optimization efforts based on features such as:
* Item Size: The physical size of the item
* Average Historical Sales: Average sales over the past 6 months
* Projected Sales: Projected sales based on seasonality from another timeseries model
* New Release: If the item was released within the last 2 months
* Shelf Life: How long the item intends to be on sale for

In [ ]:
training_df = pd.read_csv(
    "https://storage.googleapis.com/arize-assets/fixtures/demand_forecast_val.csv"
)
production_df = pd.read_csv(
    "https://storage.googleapis.com/arize-assets/fixtures/demand_forecast_prod.csv"
)

### 🤝 Upload Data to Arize: Create Arize Client
Sign up/login to your Arize account <a href="https://app.arize.com/auth/login">here</a>. Find your <a href="https://docs.arize.com/arize/api-reference/arize.pandas/client">Space and API keys</a>. Copy/paste into the cell below.

In [ ]:
SPACE_ID = "SPACE_ID"  # update value here with your Space ID
API_KEY = "API_KEY"  # update value here with your API key

arize_client = Client(space_id=SPACE_ID, api_key=API_KEY)

In [ ]:
if SPACE_ID == "SPACE_ID" or API_KEY == "API_KEY":
    raise ValueError("❌ CHANGE SPACE_ID AND/OR API_KEY")
else:
    print(
        "✅ Import and Setup Arize Client Done! Now we can start using Arize!"
    )

### 📋 Upload Data to Arize: Define Schema
Create your <a href="https://docs.arize.com/arize/sending-data-to-arize/model-schema-reference">model schema</a>. First, we'll define the features, shap values, and tags.  

In [ ]:
feature_column_names = [
    "item_size",
    "supplier_id",
    "avg_historical_sales",
    "cur_projected_sales",
    "item_new_release_flag",
    "item_stickyness_factor",
    "item_release_year",
    "shelf_life_weeks",
]

### 🪵 Upload Data to Arize: Log Training Data
Define the training schema and log the training data to Arize.

In [ ]:
# Define a Schema() object for Arize to pick up data from the correct columns for logging
training_schema = Schema(
    feature_column_names=feature_column_names,
    prediction_id_column_name="prediction_ids",
    prediction_label_column_name="predictions",
    actual_label_column_name="actuals",
)

# Logging to Arize platform using arize_client.log
training_response = arize_client.log(
    dataframe=training_df,
    model_id="supply-chain-manufacturing",
    model_version="1.0",
    model_type=ModelTypes.REGRESSION,
    environment=Environments.TRAINING,
    schema=training_schema,
)

### 🪵 Upload Data to Arize: Log Production Data
Define the training schema and log the production data to Arize.

In [ ]:
production_schema = Schema(
    feature_column_names=feature_column_names,
    prediction_id_column_name="prediction_ids",
    timestamp_column_name="prediction_ts",
    prediction_label_column_name="predictions",
    actual_label_column_name="actuals",
)

prod_response = arize_client.log(
    dataframe=production_df,
    model_id="supply-chain-manufacturing",
    model_version="1.0",
    model_type=ModelTypes.REGRESSION,
    environment=Environments.PRODUCTION,
    schema=production_schema,
)

## 🏃 Follow 'Success!' Link To Arize
Once you've successfully logged your model to Arize, follow the link to setup monitors, uncover problem areas, and more!

<strong>Note</strong>: It might take a few minutes for all the data to index in Arize, if you don't see all the data immedieatly, sit back and relax, data is on it's way!

### 🔍 In Arize: Model Setup
Now that we can see our model data in Arize, let's take a look at model performance. We'll want to catch cases of **under-forecasting** since it directly impacts customers. To do so, we'll set **Mean Error** as our default metric.

* Naviage to the 'Config' tab to configure your default metric to 'Mean Error'.
* Within the 'Config' tab, select 'Configure Baseline', and select 'Pre-Production'

<img src="https://storage.googleapis.com/arize-assets/fixtures/Industry_Use_Case/manufacturing_supply_chain_optimization_metric.png" width="1200">

### 🤖 In Arize: Monitor Setup

To prevent forecasting errors, configure monitors to get alerted right when our model deviates from expected behavior.
* Navigate to the 'Monitors' tab, select 'Setup Monitors'
* Enable 'Mean Error' and 'Mean Absolute Error' monitors, and any others that interest you!

<img src="https://storage.googleapis.com/arize-assets/fixtures/Industry_Use_Case/manufacturing_supply_chain_optimization_monitors.png" height="600">

### 📈 In Arize: Performance Tracing & Drift Tracing
Lets explore our model performance using the performance tracing tab to identify areas to improve. Since we uploaded traning and production data, we can compare the two to see how our production model.

* Click 'Add Comparison' in the 'Performance Tracing' tab
* Next to 'Dataset B', click on the the drop down menu and select training
* Click on any of the worst performing features on the 'Performance Insights' card (i.e. item_stickyness_factor)
* Click on 'View Feature Details' next to the histogram to investigate further

<img src="https://storage.googleapis.com/arize-assets/fixtures/Industry_Use_Case/manufacturing_supply_chain_optimization_drift.png" width="700">

### 📈 In Arize: Drift Tracing

The feature 'item_stickyness_factor' has a spike in drift. Let's uncover why the feature drifted.

* Click on datapoint at the highest part of the graph
* Scroll down to the 'Distribution Comparison' card

It looks like our current distribution has a spike in data in the feature slice 0-400k, which is likely the cause of performance degredation. Since this feature impacts model performance, use this information to inform feature engineering/rebuilding efforts.

<img src="https://storage.googleapis.com/arize-assets/fixtures/Industry_Use_Case/manufacturing_supply_chain_opimization_distribution.png" width="1200">

## 🚀 Continue Exploring Arize
By visually investigating model performance, we can easily identify problematic areas within our model to consider during the feature engineering and model rebuilding process.

This tutorial just scratches the surface of what Arize can do. Continue to explore the world of ML Observability with Arize to monitor, troubleshoot, and fine tune your models!

<strong>Recommended Resources:</strong>
* [Arize Community Slack](https://join.slack.com/t/arize-ai/shared_invite/zt-1is2wp3xv-SQgwwszCEeS06Sm1q4xFFw)
* [Arize Documentation](https://docs.arize.com/arize/)
* [ML Observability Course](https://courses.arize.com/)